In [ ]:
import openreview
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd

In [ ]:
venue_id = "NeurIPS.cc/2023/Conference"
username = ""
password = ""

In [ ]:
client = openreview.api.OpenReviewClient(
    baseurl='https://api2.openreview.net',
    username=username,
    password=password
)

In [ ]:
my_options = webdriver.ChromeOptions() 
my_options.add_argument(f"--headless")
driver = webdriver.Chrome(options=my_options)
driver.maximize_window()

In [ ]:
def get_ethics_reviews(forum_id):
    try:
        url = 'https://openreview.net/forum?id='
        driver.get(url + forum_id)

        reviews = []
        flags = []
        tries = 0
        while len(flags)==0:
            flags = driver.find_elements(By.XPATH,'//*[contains(text(),"Flag For Ethics Review")]')
            time.sleep(1)
            tries+=1
            if tries>=5:
                break
        for flag in flags:
            value = flag.find_element(By.XPATH, "../span")
            reviews.append(value.text)

        return reviews
    except Exception as e:
        return ["Error"]

In [ ]:
venue_group = client.get_group(venue_id)
submission_name = venue_group.content['submission_name']['value']
submissions = client.get_all_notes(invitation=f'{venue_id}/-/{submission_name}')

In [ ]:
data = []
for s in submissions:
    reviews = get_ethics_reviews(s.forum)
    data.append([s.forum] + list(reviews))

In [ ]:
df = pd.DataFrame(data)
df.to_csv(venue_id.replace("/","_") + ".csv",index=False,header=False)